In [1]:
import tensorflow as tf
import numpy as np
import random
import os
import pickle
import gc

# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Optional: disable eager execution (if using graph mode code)
#tf.compat.v1.disable_eager_execution()

# Session config
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
sess = tf.compat.v1.Session(config=session_conf)

# Set this session as default for everything that follows
# No set_session needed — use a context manager instead
with sess.as_default():
    # your model/code here
    pass

tf.__version__

2025-04-29 10:31:31.031918: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-29 10:31:31.043065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745922691.054510 2691120 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745922691.057777 2691120 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745922691.066645 2691120 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

'2.19.0'

In [2]:
os.getcwd()

'/mnt/c/Users/ansel/AppData/Local/Programs/Microsoft VS Code'

In [3]:
os.chdir('/mnt/d/GitHub/WQU-Capstone/notebooks/classes')

os.getcwd()

'/mnt/d/GitHub/WQU-Capstone/notebooks/classes'

### Import Classes

In [4]:
%load_ext autoreload
%aimport class_ForecastingTrader 
%autoreload 1

I0000 00:00:1745922693.450965 2691120 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9706 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
# series_analyser = class_SeriesAnalyser.SeriesAnalyser()
# trader = class_Trader.Trader()
# data_processor = class_DataProcessor.DataProcessor()
forecasting_trader = class_ForecastingTrader.ForecastingTrader()

In [6]:
current_path=('/mnt/d/GitHub/WQU-Capstone/data/')
with open(current_path+'yf_5_clusters_pairs_category.pickle', 'rb') as handle: 
    pairs = pickle.load(handle)
len(pairs)

20

## model train

In [7]:
n_in_set = [6, 12, 24]
hidden_nodes_set = [[30], [50]] # [[10], [20], [30], [10,10], [50]]
hidden_nodes_names = [str(nodes[0])+'*2' if len(nodes) > 1 else str(nodes[0]) for nodes in hidden_nodes_set]

In [8]:
hidden_nodes_names

['30', '50']

In [9]:
input_dim = n_in_set[-1]
for i, hidden_nodes in enumerate(hidden_nodes_set):
    model_config = {"n_in": input_dim,
                    "n_out": 1,
                    "epochs": 500,
                    "hidden_nodes": hidden_nodes,
                    "loss_fct": "mse",
                    "optimizer": "rmsprop",
                    "batch_size": 256,
                    "train_val_split": '2023-01-01',
                    "test_init": '2024-01-01',}
    models = forecasting_trader.train_models(pairs, model_config, model_type='rnn')
    # save models for this configuration
    with open('/mnt/d/GitHub/WQU-Capstone/models/lstm/models_n_in-'+str(input_dim)+'_hidden_nodes-'+hidden_nodes_names[i]+'.pkl', 'wb') as f:
        pickle.dump(models, f)

gc.collect()

I0000 00:00:1745922693.581013 2691120 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9706 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/ansel/miniconda3/envs/tf/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30)             │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,871 (15.12 KB)

 Trainable params: 3,871 (15.12 KB)

...

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.6406 - mae: 0.5453 - val_loss: 0.3370 - val_mae: 0.4775
Epoch 2/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.4621 - mae: 0.4402 - val_loss: 0.2109 - val_mae: 0.3775
Epoch 3/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.4091 - mae: 0.4123 - val_loss: 0.1626 - val_mae: 0.3285
Epoch 4/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.3867 - mae: 0.4003 - val_loss: 0.1382 - val_mae: 0.2979
Epoch 5/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.3698 - mae: 0.3910 - val_loss: 0.1232 - val_mae: 0.2784
Epoch 6/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.3550 - mae: 0.3827 - val_loss: 0.1121 - val_mae: 0.2638
Epoch 7/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.3416 - mae: 0.3749 - val_loss: 0.1031 - val_mae: 0.2512
Epoch 8/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.3296 - mae: 0.3676 - val_loss: 0.0953 - val_mae: 0.2399
Epoch 9/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.3188 - mae: 

69452

## Extra

In [ ]:
n_in_set = [6, 12, 24]
hidden_nodes_set = [[10], [20], [30], [10,10],]
hidden_nodes_names = [str(nodes[0])+'*2' if len(nodes) > 1 else str(nodes[0]) for nodes in hidden_nodes_set]

In [ ]:
for input_dim in n_in_set:
    for i, hidden_nodes in enumerate(hidden_nodes_set):
        model_config = {"n_in": input_dim,
                        "n_out": 1,
                        "epochs": 500,
                        "hidden_nodes": hidden_nodes,
                        "loss_fct": "mse",
                        "optimizer": "rmsprop",
                        "batch_size": 256,
                        "train_val_split": '2023-01-01',
                        "test_init": '2024-01-01',}
        models = forecasting_trader.train_models(pairs, model_config, model_type='rnn')
        # save models for this configuration
        with open('/mnt/f/etfs/pickle/models/rnn_models/models_n_in-'+str(input_dim)+'_hidden_nodes-'+hidden_nodes_names[i]+'.pkl', 'wb') as f:
            pickle.dump(models, f)

gc.collect()

In [ ]:
for input_dim in n_in_set:
    for i, hidden_nodes in enumerate(hidden_nodes_set):
        print('input_dim:', input_dim)

        print(i,' hidden_nodes:', hidden_nodes)

input_dim: 6
0  hidden_nodes: [10]
input_dim: 6
1  hidden_nodes: [20]
input_dim: 6
2  hidden_nodes: [30]
input_dim: 6
3  hidden_nodes: [10, 10]
input_dim: 6
4  hidden_nodes: [50]
input_dim: 12
0  hidden_nodes: [10]
input_dim: 12
1  hidden_nodes: [20]
input_dim: 12
2  hidden_nodes: [30]
input_dim: 12
3  hidden_nodes: [10, 10]
input_dim: 12
4  hidden_nodes: [50]
input_dim: 24
0  hidden_nodes: [10]
input_dim: 24
1  hidden_nodes: [20]
input_dim: 24
2  hidden_nodes: [30]
input_dim: 24
3  hidden_nodes: [10, 10]
input_dim: 24
4  hidden_nodes: [50]
